<a href="https://colab.research.google.com/github/adanfernandez/Data-Mining/blob/main/Contar_terminos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segundo ejercicio
## Preparación del entorno
En primer lugar, se procederá a la descarga del fichero limpio facilitado por el profesor.

In [ ]:
%cd /content
!mkdir limpio
!mkdir ejercicio2
%cd limpio

!gdown --id 1brYy8Tiooo2Uw--FUmPb8lpkxJGM4cnR
!unzip tweets-limpios.ndjson.zip

/content
mkdir: cannot create directory ‘limpio’: File exists
mkdir: cannot create directory ‘ejercicio2’: File exists
/content/limpio
Downloading...
From: https://drive.google.com/uc?id=1brYy8Tiooo2Uw--FUmPb8lpkxJGM4cnR
To: /content/limpio/tweets-limpios.ndjson.zip
573MB [00:05, 105MB/s]
Archive:  tweets-limpios.ndjson.zip
replace tweets-limpios.ndjson? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: tweets-limpios.ndjson   


Procedemos a la instalación de las dependencias 

In [ ]:
!pip install ndjson

# Cuerpo del ejercicio
Se procede a contar los términos. La ejecución durará apróximadamente 4 minutos.

In [ ]:
import io
import ndjson
import operator
import os

from collections import Counter
from tqdm import tqdm

def contar_palabras():
  f = io.open("tweets-limpios.ndjson", mode="r", encoding="utf-8")

  tamano_leido=0
  tamano_bloque=20*1024*1024
  tamano = os.path.getsize("tweets-limpios.ndjson")

  terminos = []
  hashtags = []
  noum_chunks = []

  with tqdm(total=tamano) as barra:
    while True:
      content = f.readlines(tamano_bloque)
      content = " ".join(content)
      tamano_leido += tamano_bloque

      if not content:
        break
      else:
        jsons = ndjson.loads(content)
        for data in jsons:
          for token in data['tokens']:
            terminos.append(token)
          for hasthag in data['hashtags']:
            if hasthag:
              hashtags.append(hasthag['text'])
          for noum_chunk in data['noun_chunks']:
            noum_chunks.append(noum_chunk)

        barra.update(tamano_bloque)

  frecuencias = Counter(terminos)
  frecuencias = sorted(frecuencias.items(),key=operator.itemgetter(1),reverse=True)
  print('Tokens', frecuencias[0:100])

  frecuencias = Counter(noum_chunks)
  frecuencias = sorted(frecuencias.items(),key=operator.itemgetter(1),reverse=True)
  print('Sintagmas nominales', frecuencias[0:100])

  frecuencias = Counter(hashtags)
  frecuencias = sorted(frecuencias.items(),key=operator.itemgetter(1),reverse=True)
  print('Hashtags', frecuencias[0:100])

contar_palabras()

2306867200it [04:21, 8822541.43it/s]


Tokens [('rt', 2041075), ('rajoy', 1523236), ('a', 1372194), ('y', 1132430), ('pp', 361552), ('bárcenas', 230547), ('q', 180553), ('dinero', 161501), ('gobierno', 152611), ('@marianorajoy', 145840), ('vía', 144398), ('✉', 133070), ('españa', 128851), ('mariano', 127967), ('prensa', 126483), ('cierra', 126024), ('presidente', 125639), ('o', 120534), ('papa', 119008), ('mato', 105877), ('d', 101515), ('negro', 100363), ('millones', 97956), ('psoe', 93378), ('rueda', 92638), ('ana', 91544), ('preguntas', 89528), ('euros', 83702), ('partido', 82022), ('país', 78050), ('cospedal', 76571), ('corrupción', 76152), ('yocreoenrajoy', 73499), ('dimisión', 67803), ('mañana', 64764), ('sobres', 62800), ('caso', 61776), ('aparece', 59907), ('renta', 59841), ('cuentas', 58184), ('rubalcaba', 57140), ('telón', 54332), ('rajoynocontesta', 53722), ('@el_pais', 52377), ('m', 52160), ('renuncia', 51610), ('política', 51435), ('años', 51330), ('declaración', 49538), ('señor', 49443), ('parados', 48904), ('

Tras esto, se pueden sacar varias conclusiones acerca de las palabras vacías.
Por un lado, la principal palabra vacía sería *rt*, el cual sale tanto en los tokens como en los sintagmas nominales y se refiere a los Retweets hechos a tweets originales. Es decir, gente que comparte un tweet.
Después se podrían añadir otras tales como conjunciones -*y*-, pronombres -*se*- o palabras conformadas por un solo carácter. También se debería de estudiar el caso de los emoticonos. Por ejemplo, tenemos el caso de los sobres -✉-, que sí podrían que tenerse en cuenta en lo que respecta al tweet.